# Рахматуллин Амир

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# https://openflights.org/data.html

url_airport = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
url_routers = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'

df_raw_airport = pd.read_csv(url_airport, index_col=0)
display(df_raw_airport.head(5))
df_raw_routers = pd.read_csv(url_routers)
display(df_raw_routers.head(10))

In [ ]:
columns_airport = ['Airport ID', 'Name', 'City',
                   'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone',
                   'DST', 'Tz database time zone', 'Type', 'Source']

columns_routers = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 
                   'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']

df_raw_airport = pd.read_csv(url_airport, names = columns_airport, index_col=0)
display(df_raw_airport.head(3))
df_raw_routers = pd.read_csv(url_routers, names = columns_routers)
display(df_raw_routers.head(10))


# Задание 1

Добавить столбец <code>Continent</code> с данными, которые будут содержать название континента.

In [ ]:
#pip install a-world-of-countries

import awoc

my_world = awoc.AWOC()

def country_to_continent(name):
    try:
        continent = my_world.get_country_continent_name(name)
    except NameError:
        continent = np.nan
    return continent


dict_country_error = {
    "Cote d'Ivoire": 'Ivory Coast',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Congo (Brazzaville)':'Republic of the Congo', 
    'Burma':'Myanmar',
    'Cocos (Keeling) Islands': 'Cocos Islands'
}

df_raw_airport['Country'] = df_raw_airport['Country'].replace(dict_country_error)
df_raw_airport['Continent'] = df_raw_airport['Country'].map(country_to_continent)
df_raw_airport[df_raw_airport['Continent'].isna()]

dict_teritor_error = {
    'West Bank': 'Asia', 
    'Midway Islands': 'Oceania', 
    'French Guiana': 'South America', 
    'Martinique': 'North America',
    'Guadeloupe': 'North America', 
    'Virgin Islands': 'North America', 
    'Norfolk Island': 'Oceania', 
    'Johnston Atoll': 'Oceania',
    'Svalbard': 'Europe', 
    'Wake Island': 'Oceania'}

df_raw_airport['Continent'] = df_raw_airport['Continent'].fillna(df_raw_airport['Country'].map(dict_teritor_error))

df_raw_airport.head(7)



# Задание 2

Из списка континентов случайно выбрать континент и срезать таблицу по названию континента и аэропорту имеющему код IATA (имеется в виду - для выбранного континента только аэропорты с кодом IATA). Каждый раз при выполнении кода континент случайно выбирается.

In [ ]:
random_continent = np.random.choice(['Europe', 'Asia', 'North America', 'South America', 'Africa', 'Oceania', 'Antarctica'])

print('We randomly chose', random_continent)


df = df_raw_airport[df_raw_airport["Continent"] == random_continent]


df = (df.loc[df['IATA'] != '\\N'])


display(df.head(10))


# Задание 3

На основе таблицы связи аэропортов создать таблицу с кодом аэропорта, количеством взлетов и количеством посадок.

In [ ]:
takeoffs = df_raw_routers.groupby("Source airport")['Source airport ID'].count()


landings = df_raw_routers.groupby("Destination airport")['Destination airport ID'].count()


df_table = pd.concat([takeoffs, landings], axis = 1, sort = True, keys = ['Takeoffs', 'Landings']).fillna(0)
df_table

# Задание 4

Используя в качестве ключа код аэропорта слить данные для отобранного континента и собрать таблицу - страна, название аэропорта, количество взлет/посадок (сумма взлетов и посадок)

In [ ]:
if random_continent == 'Antarctica':
    print('No data of Antarctica in df_raw_routers')
else:
    wwww = pd.merge(df, df_table, left_on = 'IATA', right_index = True, how='inner')

    wwww['Sum'] = wwww['Takeoffs'] + wwww['Landings']

    table4 = wwww.pivot_table(index = ['Country','Name'], values = 'Sum')

    display(table4)


# Задание 5

1. Найти коды аэропортов для которых нет взлет/посадок в таблице аэропортов.
2. Найти коды аэропортов для которых нет названий в таблице связей аэропортов.

In [ ]:
un_routers = df_raw_routers["Source airport"].unique()

mask_1 = df_raw_airport["IATA"].isin(un_routers)


display(pd.DataFrame(df_raw_airport["IATA"][~mask_1].unique()).set_axis(['Airports not in routers table'], axis='columns'))


un_airports = df_raw_airport["IATA"].unique()

mask_2 = df_raw_routers['Source airport'].isin(un_airports)

pd.DataFrame(df_raw_routers["Source airport"][~mask_2].unique()).set_axis(['Airports not in airports table'], axis='columns')


# Задание 6

Для таблицы в задании 4, для трех крупнейших аэропортов для каждой страны, собрать ряд - первый индекс: название страны, второй индекс: название аэропорта, значение: количество взлет посадок. Первый индекс упорядочен по названию страны (по алфавиту), а назание аэропорта по количеству взлет посадок от большего к меньшему.

In [ ]:
if random_continent == 'Antarctica':
    print('No data of Antarctica in df_raw_routers')
else:
    ttt = table4.reset_index('Country')

    for_every_country = pd.DataFrame(ttt.groupby('Country')['Sum'].nlargest(3))

    display(for_every_country)

# Задание 7

Для каждого континента отобрать не более 3 стран с наибольшим количеством взлет посадок и нанести их на географическую карту. 

In [ ]:
if random_continent == 'Antarctica':
    print('No data of Antarctica in df_raw_routers')
else: 
    three_countries = pd.DataFrame(table4.groupby("Country")["Sum"].sum().nlargest(3))

    temp = three_countries.reset_index()


    list = []

    for i in range(0,3):
        list.append(my_world.get_country_ISO3(temp['Country'][i]))

    from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
  

    init_notebook_mode(connected = True)

    data = dict(type = 'choropleth',
  
            locations = list,
            
            locationmode = 'ISO-3',
              
            colorscale = 'Portland',
            
            text = [temp['Country'][0], temp['Country'][1], temp['Country'][2]],
            
            z = [temp['Sum'][0], temp['Sum'][1], temp['Sum'][2]],
                
            colorbar = {'title': 'Sum of takeoffs and landings'})

    choromap = go.Figure(data = [data])

    iplot(choromap)

# Задание 8

Для каждого континента отобрать 5 наиболее крупных аэропортов с наибольшим количеством взлет посадок и нанести их на географическую карту. 

In [ ]:
if random_continent == 'Antarctica':
    print('No data of Antarctica in df_raw_routers')
else:
    biggest = wwww.pivot_table(values = 'Sum', index = ['IATA','Name', 'City']).nlargest(5, 'Sum')
    
    rr = biggest.reset_index()
    

    mas = []
    for i in range(0,5):
        mas.append(rr['Name'][i])
    
    ppp = df_raw_airport.loc[df_raw_airport['Name'].isin(mas)]
    
    ooo = pd.pivot_table(ppp, index = 'City', values = ['Latitude', 'Longitude'])
    
    xxx = pd.merge(rr, ooo, left_on = 'City', right_index = True)
    display(xxx)
    
    fig = px.scatter_mapbox(xxx, lat="Latitude", lon="Longitude", hover_name="Name", hover_data=["Sum"],
                        color_discrete_sequence=["red"], zoom=2, height=500)

    fig.update_layout(mapbox_style="carto-positron")

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig.show()
    
    

# Задание 9

Для наиболее крупных аэропортов (задание 8) построить маршрутную транспортную сеть и нанести их на географическую карту, линия должна отображать интенсивность взаимосвязи (прозрачность, толщина). 

In [ ]:
if random_continent == 'Antarctica':
    print('No data of Antarctica in df_raw_routers')
else:
    gg = []
    for i in range(0,5):
        gg.append(ppp.reset_index()['IATA'][i])

    table9 = df_raw_routers.loc[(df_raw_routers['Source airport'].isin(gg)) & (df_raw_routers['Destination airport'].isin(gg))][['Source airport', 'Destination airport']]

    def get_pair(i, j):
        return table9.loc[(table9['Source airport'] == i) & (table9['Destination airport'] == j)]

    u = pd.DataFrame()

    for i in gg:
        for j in gg:
            cc = get_pair(i,j)
            u = pd.concat([u,cc], axis = 0)

    lats_fly = []
    lons_fly = []


    for i in u['Source airport']:
        lats_fly.append(xxx.set_index('IATA').loc[i]["Latitude"])

    for i in u['Source airport']:
        lons_fly.append(xxx.set_index('IATA').loc[i]["Longitude"])

    fig = go.Figure(go.Scattermapbox(
        mode = "markers+lines",
        lon = lons_fly,
        lat = lats_fly,
        marker = {'size': 10}))


    fig.update_layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        mapbox = {
            'center': {'lon': 10, 'lat': 10},
            'style': "stamen-terrain",
            'zoom': 2})

    fig.show()

